In [2]:
import sys
import numpy as np
import pandas as pd

import matplotlib as mpl
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("public_150k_plus_230101.csv")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 968531 entries, 0 to 968530
Data columns (total 53 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   LoanNumber                   968531 non-null  int64  
 1   DateApproved                 968531 non-null  object 
 2   SBAOfficeCode                968531 non-null  int64  
 3   ProcessingMethod             968531 non-null  object 
 4   BorrowerName                 968527 non-null  object 
 5   BorrowerAddress              968517 non-null  object 
 6   BorrowerCity                 968519 non-null  object 
 7   BorrowerState                968518 non-null  object 
 8   BorrowerZip                  968518 non-null  object 
 9   LoanStatusDate               930864 non-null  object 
 10  LoanStatus                   968531 non-null  object 
 11  Term                         968531 non-null  int64  
 12  SBAGuarantyPercentage        968531 non-null  int64  
 13 

In [25]:
df2 = df.loc[df['ProcessingMethod'] == "PPP"].value_counts()

df2

LoanNumber  DateApproved  SBAOfficeCode  ProcessingMethod  BorrowerName                                 BorrowerAddress                   BorrowerCity  BorrowerState  BorrowerZip  LoanStatusDate  LoanStatus    Term  SBAGuarantyPercentage  InitialApprovalAmount  CurrentApprovalAmount  UndisbursedAmount  FranchiseName            ServicingLenderLocationID  ServicingLenderName                        ServicingLenderAddress  ServicingLenderCity  ServicingLenderState  ServicingLenderZip  RuralUrbanIndicator  HubzoneIndicator  LMIIndicator  BusinessAgeDescription             ProjectCity  ProjectCountyName  ProjectState  ProjectZip  CD     JobsReported  NAICSCode  Race        Ethnicity          UTILITIES_PROCEED  PAYROLL_PROCEED  MORTGAGE_INTEREST_PROCEED  RENT_PROCEED  REFINANCE_EIDL_PROCEED  HEALTH_CARE_PROCEED  DEBT_INTEREST_PROCEED  BusinessType             OriginatingLenderLocationID  OriginatingLender                          OriginatingLenderCity  OriginatingLenderState  Gender      Vete